In [1]:
import numpy as np
import pandas as pd
import requests

In [3]:
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

data = {
    'grant_type': 'password',
    'username': username,
    'password': password
}

In [4]:
#create an informative header for your application
headers = {'User-Agent': 'namehere/0.0.1'}

res = requests.post(
    'https://www.reddit.com/api/v1/access_token',
    auth=auth,
    data=data,
    headers=headers)

print(res)

<Response [200]>


In [5]:
#retrieve access token
token = res.json()['access_token']

In [6]:
headers['Authorization'] = f'bearer {token}'
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).status_code == 200

True

In [7]:
base_url = 'https://oauth.reddit.com/r/'

In [8]:
df = pd.read_csv(r'../datasets/reddit_dataframe.csv')

How to use requests.get() parameters:</br>
https://www.w3schools.com/PYTHON/ref_requests_get.asp

`old_df` is the existing df to append to

In [9]:
def get_posts(old_df):
    for s in ['AntiWork', 'Jobs']:
        # allow 'after' param to be default first pass
        after_index = ''
        sub_df = pd.DataFrame()
        for f in range(0, 1100, 100):
            params = {
                    "limit": 100,
                    'count': f,
                    "after": after_index
            }

            res = requests.get(base_url + s, 
                                   headers = headers, 
                                   params = params)

            res = res.json()
            post_count = len(res['data']['children'])
            text = [res['data']['children'][i]['data'] for i in range(2, post_count)]
            text = [t for t in text if t != '']
            # convert text to dataframe format
            temp_df = pd.DataFrame(text)
            # store temp_df info to subreddit-scale dataframe
            sub_df = pd.concat([temp_df, sub_df], axis = 0)
            # update after_index
            after_index = res['data']['after']
            # break loop if the post limit is reached
            if after_index is None:
                break
                
        old_df = pd.concat([old_df, sub_df], axis = 0)
    df[['subreddit', 'selftext']] = df[['subreddit', 'selftext']].dropna()
    old_df = old_df.drop_duplicates(subset = ['selftext'], keep = 'first')
    old_df.index = range(len(old_df))
    return old_df

In [10]:
df = get_posts(df)
df.to_csv('../datasets/reddit_dataframe.csv', index_label = False)
df.shape

(7155, 116)

In [15]:
# the classes are pretty even
df['subreddit'].value_counts(normalize = True)

jobs        0.528236
antiwork    0.471764
Name: subreddit, dtype: float64